<a href="https://colab.research.google.com/github/Hemachandran-D45/vectordb/blob/main/Langchian_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install langchain-huggingface -q

It is new library announced by hugging face

In [ ]:
!pip install huggingface_hub
!pip install langchain
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

Hugging face hub has repositories of various models which you can specifically call from hugging face
Transformer important cuz if i probably go ahead and search respect any model in hugging face

In [ ]:
!pip install mistral_inference -q


In [ ]:
!pip install langchain_core langchain_huggingface

Calling a model in two ways
1) Directly downloading a model in local
2) Can call this model through endpoint

Hugging Face Endpoint

Access Hugging Face Model with API (Hugging Face Hub)
There are two ways to use this class. You can specify the model with repo_id parameter. Those endpoints use the serverless API which is pariculary beneficial to people using pro account. Still, regular user can already have access to fair amount of request with their Hugging Face Token in the environment where they are executing the code.


In [5]:
from langchain_huggingface import HuggingFaceEndpoint

In [6]:
#Environment for Secret Key

from google.colab import userdata
sec_key = userdata.get("Langchain_Testing")

In [7]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key

based on the authentication we will be able to call any model from hugging face

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=128,
    temperature=0.7,
    token=sec_key
)

In [ ]:
llm.invoke("I want to start a restaurant for Indian food, Suggest a fancy name ")

In [ ]:
try:
  response = llm.invoke("What is machine learnig")
  print(response)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from re import template
from langchain import PromptTemplate, LLMChain

question = "I want to oepn an restaurant for Indian Food, suggest a fancy name."
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
print(prompt)

In Langchain we have prompt template which actually help us to create your own custom and we can entirely use kind of RAG application. Retrieval Argument Generation, where we can provide our question

In [ ]:
llm_chain=LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question)
print(response)

llm is nothing but that model mistral , prompt is our prompt.

Hugging Face Pipeline

Among transformer,the pipeline is the most versatile tool in the Hugging Face  toolbox. Langchain being designed primarily to address RAG an Agent use cases, the scope of the pipeline here is reduced to the following text-centric task. "text-generation","text2text-generation","summarization","translation". Models can be loaded directly with the from_model_id method

if we want to download entire model in local environment that time we can use Hugging Face Pipeline but the one disadvantage is Very Big LLM model or multi model cannot be downloaded locally due to RAM issue

In [13]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM,AutoTokenizer, pipeline

Automodel for causal Lm - we can import any model just by using the function pretrained , for every model there will be tokenizer (take a input and convert that into some embedding vectors)

In [ ]:
model_id="gpt2"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [15]:
#kind of task is the first parameter

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=100)
HFP = HuggingFacePipeline(pipeline=pipe)

In [16]:
HFP  ## model

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d3dc94c8a30>)

In [17]:
HFP.invoke("What is Machine Learning")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"What is Machine Learning?\n\nMachine learning is a tool for learning, which means that it doesn't require any programming skill. It lets you develop predictive models that can lead to better outcomes. While using Machine Learning is certainly a viable alternative to Artificial Intelligence as a general purpose software platform, it can be very time consuming and takes a lot of time as well. The primary goal of ML projects has been to help provide a robust, scalable, and extensible programming language for all applications in robotics, aerospace, and medical"

In [18]:
import torch
print(torch.cuda.is_available())

False


In [19]:
##Use the Hugging Face Pipeline with CPU

gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=-1,
    pipeline_kwargs = {"max_new_tokens":100}
  )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
from langchain_core.prompts import PromptTemplate

template = "Question : {question}"
prompt = PromptTemplate.from_template(template)

In [21]:
chain=prompt|gpu_llm #another way of creating LLM Chain

In [24]:
question = "What is Artificial Intelligence"
chain.invoke({"question" : question})

'Question : What is Artificial Intelligence?\nThe core of Artificial Intelligence is called "Artificial Intelligence" or AI. This term refers to some of the techniques and concepts that we used to learn about Artificial Intelligence (AI).\nArtificial Intelligence uses the techniques and concepts associated with robotics (Birds and dinosaurs), visual, acoustic and motion sensors, actuators, and other types of tools. Most recent innovations include quantum computer, distributed systems, deep learning, computer graphics, and artificial intelligence, which are all ways of helping create more'